This notebook goes out and gets ITIS information for the set of species in the ESA petitioning process. It uses the ITIS TSN identifiers that we pulled out previously by either scraping the linked ECOS web pages or fetching TESS data. In this process, we'll be looking for interesting anomalies that may need further investigation:

* Any cases where a specified TSN shows up as invalid from ITIS
* Any cases where there is a mismatch between ITIS TSNs in the ECOS scrape and TESS data (indicating some potential issue in the FWS data system)

In [1]:
from bis2 import dd
from IPython.display import display
from datetime import datetime
from bis import itis
import requests

In [2]:
bisDB = dd.getDB("bis")
esaWPSpecies = bisDB["FWS ESA Work Plan Species"]

First, we check just to see if there were any cases where there is a mismatch between the TSNs that we have onboard the items at this point. If there is an issue, then we need to go back and evaluate the data further to see where there is a misalignment in the FWS data system.

In [3]:
for record in esaWPSpecies.find({"$and":[{"TESS.TSN":{"$exists":True}},{"ECOS Scrape.TSN":{"$exists":True}}]}):
    if record["ECOS Scrape"]["TSN"] != record["TESS"]["TSN"]:
        display (record)

I opted to use the latest processor I built for retrieving ITIS information based on scientific name. This runs a search against the ITIS Solr API and returns all available information from the resultset starting point. If a search returns an invalid or unaccepted ITIS record on name, the function follows accepted taxonomy to pull all viable records together into a list of documents for later processing and analysis.

In [6]:
for record in esaWPSpecies.find():
    scientificName = record["Submitted Data"]["Scientific Name"]
    if "TESS" in record.keys() and "SCINAME" in record["TESS"].keys():
        scientificName = record["TESS"]["SCINAME"]
    else:
        if "ECOS Scrape" in record.keys() and "Scientific Name" in record["ECOS Scrape"].keys():
            scientificName = record["ECOS Scrape"]["Scientific Name"]
    
    print (esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"ITIS":itis.checkITISSolr(scientificName)}}))